# Show top/worst _n_% of the sentences filtered by freq of word counts (in HT minus in MT) from _analysis table_

In [3]:
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager
from pprint import pprint

from jieba_postag_converter import get_postag_dict

def get_data_as_list(fpath) -> list:
    data = []
    with open(fpath, "r", encoding="utf-8-sig") as f:
        for line in f:
            line = line.replace("\n","").split(" ")
            data.append(line)
    return data

def get_diff_sentences(df,corpus="mpdd", situation='apology', sen_type='query', diff_type='del', relation='inferior', percentile=75, top=True, thirdcol_obj='primitive form'):
    def count(x):
        return x.count() 
    df = df[df['corpus'].isin([corpus])&df['situation'].isin([situation])&df['sentence type'].isin([sen_type])&df['difference type'].isin([diff_type])&df['upper relation'].isin([relation])]
    unaligned_count = df.groupby(['corpus', 'situation', 'method', 'sentence type', 'difference type','line','upper relation'])['index'].apply(count).reset_index()
    unaligned_count = unaligned_count.rename(columns={'index': 'count'})
    
#     unaligned_count['quantile'] = unaligned_count.groupby(['corpus', 'situation', 'method', 'sentence type', 'difference type',])['count'].transform(lambda group: np.quantile(group,quantile))
#     print(unaligned_count)
#     if top==True:
#         diff_df = unaligned_count.groupby(['corpus', 'situation', 'method', 'sentence type', 'difference type','line']).filter(lambda group: group['count']>=group['quantile']).reset_index()
#     else:
#         diff_df = unaligned_count.groupby(['corpus', 'situation', 'method', 'sentence type', 'difference type','line']).filter(lambda group: group['count']<=group['quantile']).reset_index()

    unaligned_count.set_index('line',inplace=True)
    translated_dict = unaligned_count[unaligned_count['method']=='translated']['count'].to_dict()
    rewrited_dict = unaligned_count[unaligned_count['method']=='rewrited']['count'].to_dict()
    calced_dict={}
    for i in range(max(max(translated_dict),max(rewrited_dict))+1):
        if (i in translated_dict) and (i in rewrited_dict):
            calced_dict[i] = rewrited_dict[i] - translated_dict[i]
        elif (i not in translated_dict) and (i in rewrited_dict):
            calced_dict[i] = rewrited_dict[i]
        elif (i  in translated_dict) and (i not in rewrited_dict):
            calced_dict[i] = - translated_dict[i]
        else:
            pass
    values = list(calced_dict.values())
    threshold = np.percentile(np.array(values),percentile)
    calced_dict = dict(sorted(calced_dict.items(), key=lambda i: i[1], reverse=True))
#     print(calced_dict)
    method_type = ['original','translated','rewrited']
#     sentence_type = ['query', 'res']
    sentence_type = [sen_type]
    dirs = ['../mrphdata/']
    print(f'{corpus} {situation} {diff_type}')
    for line, val in calced_dict.items():
        if top==True and val < threshold:
            continue
        if top==False and val > threshold:
            continue
        for t in sentence_type:
            for m in method_type:
                for d in dirs:
                    path =f'{d}/{corpus}/{situation}/{m}_{t}'
                    with open(path, 'r', encoding='utf_8_sig')as f:
                        for i,sen in enumerate(f):
                            if i == line:
                                print('[{0:^5}] {1:<11} {2:<6}: {3}'.format(i,m,t,sen[:-1]))
        unaligned_tr = df[(df['line'].isin([line]))&(df['method'].isin(['translated']))]['word'].to_list()
        unaligned_re = df[(df['line'].isin([line]))&(df['method'].isin(['rewrited']))]['word'].to_list()
#             print(type(df))
        print('{0:<11}: {1}'.format('translated',' '.join(unaligned_tr)))
        print('{0:<11}: {1}'.format('rewrited',' '.join(unaligned_re)))
        print()
#     [diff_df['method']=='translated']['count'].to_dict()
#     print(translated_count)
#     for  g, item in diff_df:
#         print(g)
#         print(item)
#     return 0

In [22]:
t_path = "analysis_table.csv"
# t_path = "analysis_table_upper.csv"

df = pd.read_csv(t_path)

get_diff_sentences(df,'cejc','request','query','del','inferior',30,top=True)

cejc request del
[  0  ] original    query : そう です 。 すいません 。 ちょっと なんか もう 申し訳 な いっす け ど 。 お 願い し ます 。
[  0  ] translated  query : 是 的 ， 它 是 。 。 对不起 ， 我 不 知道 。 我 很 抱歉 ， 但是 ... 。 求 你 了
[  0  ] rewrited    query : 不好意思 给 你 添麻烦 了 。
translated : 
rewrited   : すいません ちょっと もう いっす け ど 願い し ます

[  1  ] original    query : お 願い し ます 。 それ いい です ね 。 そ したら 。
[  1  ] translated  query : 求 你 了 。 这 是 很好 的 。 。 然后 。
[  1  ] rewrited    query : 麻烦 你 了 。 听 起来 不错 啊 。 也就是说 ...
translated : それ いい です
rewrited   : ます それ いい です

[  4  ] original    query : 今 フォロー も し ます 。 だ 連絡 だけ ちょっと 申し訳 な いっす け ど 。 お 願い し ます 。 で (. 大槻 先生 に あの たぶん きょうか あす たぶん きょう あす に かけて 運営 に 連絡 して いただいて 。
[  4  ] translated  query : 我 现在 也 会 跟进 的 。 。 我 只是 打电话 告诉 你 ， 我 很 抱歉 。 。 求 你 了 。 在 （ 。   大 槻 老师 + 那 也许 今天 或 明天 + 也许 今天 或 明天 ， 并 联系 了 管理 部门 。
[  4  ] rewrited    query : 现在 有 比较 注意 他 的 状况 了 。 不好意思 ， 那 就 再 麻烦 你 跟 他 联络 了 。 诶 ... 大概 今天 或是 明天 会 再 跟 大 槻 老师 联络 就 可以 了 。
translated : し ます 先生 連絡 して いただいて
rewrited   : ます ちょっと な いっす け ます

[ 54  ] original

In [18]:
get_diff_sentences(df,'cejc','request','query','del','peer',70,top=True)

cejc request del
[ 38  ] original    query : もらい ます もらい ます 。 こ れ そう ね 。 この あの 赤い の 取って くれ ない ? 。
[ 38  ] translated  query : 我 要 了 ， 我 要 了 。 。 这个 + 那个 。 。 你 能 把 这个 + 那个 + 红色 的 递给 我 吗 ？
[ 38  ] rewrited    query : 好好 好 我 拿 ， 你 等 我 一下 。 诶 ... 你 能 不能 帮 我 拿 一下 红色 的 那 一个 ？
translated : そう
rewrited   : ます もらい ます れ この あの

[ 35  ] original    query : はい 。 お 願い し ます 。
[ 35  ] translated  query : 是 的 ， 我 知道 。 求 你 了
[ 35  ] rewrited    query : 好 的 。
translated : 
rewrited   : お 願い し ます

[ 16  ] original    query : なんで 皆さん よろしく お 願い し ます 。 九 月 は 毎週 あり ます んで 。
[ 16  ] translated  query : 你们 为什么 不 + 谢谢你们 的 支持 。 。 9 月份 每周 都 会 有 一个 。
[ 16  ] rewrited    query : 那么 ， 九月 的 每个 星期 就 再 麻烦 各位 了 。
translated : よろしく お 願い し
rewrited   : お 願い ます 月 毎週 ます んで

[ 26  ] original    query : なんとか が あった ので えー と 悪 天候 時 は 無理 し ない 。
[ 26  ] translated  query : 天气 不好 的 时候 ， 我 不会 勉强 自己 。
[ 26  ] rewrited    query : 诶 ... 如果 天气 真的 很 的话 ， 就 真的 不要 勉强 了 。
translated : 
rewrited   : ので し ない

[ 41  ] original    query : あ 。 そう です

In [19]:
get_diff_sentences(df,'cejc','request','query','del','superior',50,top=True)

cejc request del
[ 71  ] original    query : じゃあ お ばあちゃん と か に も お 願い すれば いい じゃ ん 。
[ 71  ] translated  query : 那 你 为什么 不 直接 问 + 奶奶 什么 的 ？
[ 71  ] rewrited    query : 那 你 干嘛 不 直接 拜 讬 奶奶 就 好 了 ？
translated : お 願い
rewrited   : じゃあ ばあちゃん

[  3  ] original    query : オッケオッケ 。 が それ に ン ね 智樹 の フォロー と か そっち は お 願い ね 。
[  3  ] translated  query : 好 吧 + 好 吧 。 。 我 不 知道 该 说 什么 。
[  3  ] rewrited    query : OK ！ 啊 对了 ， 就 麻烦 你 再 多 注意 一下 智树 的 状况 了 。
translated : それ 智樹 お 願い
rewrited   : 

